In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import  RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import cross_val_score
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,Binarizer
from sklearn.model_selection import learning_curve
from collections import Counter
# from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,f1_score
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)







In [4]:


#平均数填充，归一化
def mean_fill(data,feature):
    array=data.loc[:,feature].values.reshape(-1,1)
    SUM=0
    count=0
    for i in range(len(array)):
        if array[i,0]!='(null)':
            array[i,0]=float(array[i,0])
            count=count+1
            SUM=SUM+array[i,0]
    data.loc[data[feature]=='(null)',feature]=SUM/count
    data[feature] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[[feature]]))
    return data

#判断是否是中文、NULL、(null),删除行
def if_ch(data,feature):
    array=data.loc[:,feature].values.reshape(-1,1)
    for i in range(len(array)):
        for ch  in  str(array[i,0]):
            if u'\u4e00' <= ch <= u'\u9fff' :
                data.iat[i,list(data.columns).index(feature)]='True'
        if str(array[i,0])=='NULL' or str(array[i,0])=='(null)':
            data.iat[i, list(data.columns).index(feature)] = 'True'
    data.drop(data.loc[data[feature]=='True'].index, inplace=True)
    return data

#判断是否为整数，删除非整数的行
def isIntSeriously(data,feature):
    array = data.loc[:, feature].values.reshape(-1, 1)
    for i in range(len(array)):
        n = float(array[i,0])
        if n.is_integer() and str(array[i,0]).count('.') == 0:
            pass
        else:
            data.iat[i, list(data.columns).index(feature)] = 'True'
    data.drop(data.loc[data[feature] == 'True'].index, inplace=True)
    return data

#base_info
def get_base_info(data):
    data['Stat_Name']=LabelEncoder().fit_transform(data['Stat_Name'])
    data.loc[(data['AGE']>=16)&(data['AGE']<=29), 'AGE'] = 1
    data.loc[data['AGE']==0, 'AGE'] = 1
    data.loc[data['AGE'] != 1, 'AGE'] = 0
    # 读取高危地
    data_gaowei = pd.read_excel('./高危地.xlsx')
    array = data.loc[:, 'ID_NBR_CITY'].values.reshape(-1, 1)
    for i in range(len(array)):
        if array[i, 0] in list(data_gaowei['city_name']):
            data.iat[i, list(data.columns).index('ID_NBR_CITY')] = 1
        else:
            data.iat[i, list(data.columns).index('ID_NBR_CITY')] = 0
    data['CHANNEL_LVL1_NAME_2014'] = LabelEncoder().fit_transform(data['CHANNEL_LVL1_NAME_2014'])
    data.loc[data['REAL_CUST_FLG']=='(null)','REAL_CUST_FLG']=-1
    data.loc[data['SAME_ID_ASSETS_FLG'] == '(null)', 'SAME_ID_ASSETS_FLG'] = -1
    data['innet_month']=MinMaxScaler(feature_range=(0,1)).fit_transform((data[['innet_month']]))
    data.loc[data['MKT_STAR_GRADE_NAME'] == '(null)', 'MKT_STAR_GRADE_NAME'] = -2
    mean_fill(data,'CDMA_CNT')
    data['ITV_CNT']=MinMaxScaler(feature_range=(0,1)).fit_transform((data[['ITV_CNT']]))
    data['FIX_CNT']=MinMaxScaler(feature_range=(0,1)).fit_transform((data[['FIX_CNT']]))
    mean_fill(data,'SAME_ID_C_CNTS')
    return data

#consume_behavior
def get_consume_behavior(data):
    data.loc[data['is_agree_flg'] == '(null)', 'is_agree_flg'] = -1
    data.loc[data['prom_amt'] == 5, 'prom_amt'] = 1
    data.loc[data['prom_amt'] == 19, 'prom_amt'] = 1
    data.loc[data['prom_amt'] == 29, 'prom_amt'] = 1
    data.loc[data['prom_amt'] != 1, 'prom_amt'] = 0
    mean_fill(data, 'T0_FIN_AMT')
    mean_fill(data, 'T1_FIN_AMT')
    data['R3_FIN_AMT'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['R3_FIN_AMT']]))
    mean_fill(data, 'PROM_T0_FIN_AMT')
    return data

#calling_behavior
def get_calling_behavior(data):
    mean_fill(data,'MAXDAY_CALL_CNT')
    data['OPP_NBRS']=MinMaxScaler(feature_range=(0,1)).fit_transform((data[['OPP_NBRS']]))
    data['OPP_NBRS_CTC_PCT']=MinMaxScaler(feature_range=(0,1)).fit_transform((data[['OPP_NBRS_CTC_PCT']]))
    data.loc[(data['CALL_DIS'] >= 0.84) & (data['CALL_DIS'] <= 1), 'CALL_DIS'] = 1
    data.loc[data['CALL_DIS'] != 1, 'CALL_DIS'] = 0
    data.loc[(data['O_Call_Cnt'] ==0) | (data['O_Call_Cnt'] == 1), 'O_Call_Cnt'] = 1
    data.loc[data['O_Call_Cnt'] != 1, 'O_Call_Cnt'] = 0
    data['O_Call_Dur'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['O_Call_Dur']]))
    mean_fill(data, 'T0_CALL_DAYS')
    mean_fill(data, 'T0_CALL_CNT')
    mean_fill(data, 'T0_CALL_DUR')
    mean_fill(data, 'T1_CALL_DAYS')
    mean_fill(data, 'T1_CALL_CNT')
    mean_fill(data, 'T1_CALL_DUR')
    data.loc[(data['R3_CALL_DAYS'] == 0) | (data['R3_CALL_DAYS'] == 1)| (data['R3_CALL_DAYS'] == 2)| (data['R3_CALL_DAYS'] == 3), 'R3_CALL_DAYS'] = 1
    data.loc[data['R3_CALL_DAYS'] != 1, 'R3_CALL_DAYS'] = 0
    data['R3_CALL_CNT'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['R3_CALL_CNT']]))
    data['R3_CALL_DUR'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['R3_CALL_DUR']]))
    mean_fill(data, 'T0_O_AVG_CALL')
    data['T0_O_CALL_60'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['T0_O_CALL_60']]))
    data['T0_O_CALL_120'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['T0_O_CALL_120']]))
    data['T0_O_CALL_300'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['T0_O_CALL_300']]))
    data['T0_O_CALL_300s'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['T0_O_CALL_300s']]))
    data_gaowei = pd.read_excel('./高危地.xlsx')
    array = data.loc[:, 'FIRST_CALL_PLACE'].values.reshape(-1, 1)
    for i in range(len(array)):
        if array[i, 0] in list(data_gaowei['city_name']):
            data.iat[i, list(data.columns).index('FIRST_CALL_PLACE')] = 1
        else:
            data.iat[i, list(data.columns).index('FIRST_CALL_PLACE')] = 0
    return data

#sms_behavior
def get_sms_behavior(data):
    data['R3_SMS_CNTS'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['R3_SMS_CNTS']]))
    data['R3_SMS_DAYS'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['R3_SMS_DAYS']]))
    data['T0_SMS_CNTS'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['T0_SMS_CNTS']]))
    data['T0_SMS_DAYS'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['T0_SMS_DAYS']]))
    return data

#internet_behavior
def  get_internet_behavior(data):
    data.loc[(data['slient_days'].astype(int) >= 0) & (data['slient_days'].astype(int) <= 30 ), 'slient_days'] = 1
    data.loc[data['slient_days'] != 1, 'slient_days'] = 0
    data.loc[(data['t0_ONLINE_DAY'] == 0) | (data['t0_ONLINE_DAY'] == 1) | (data['t0_ONLINE_DAY'] == 2) | (data['t0_ONLINE_DAY'] == 3), 't0_ONLINE_DAY'] = 1
    data.loc[data['t0_ONLINE_DAY'] != 1, 't0_ONLINE_DAY'] = 0
    data.loc[data['t1_ONLINE_DAY'] == 0,'t1_ONLINE_DAY'] = 1
    data.loc[data['t1_ONLINE_DAY'] != 1, 't1_ONLINE_DAY'] = 0
    data.loc[(data['R3_ONLINE_DAY'].astype(int) >= 0) & (data['R3_ONLINE_DAY'].astype(int)<= 10), 'R3_ONLINE_DAY'] = 1
    data.loc[data['R3_ONLINE_DAY'] != 1, 'R3_ONLINE_DAY'] = 0
    mean_fill(data, 'T0_ACT_DAYS')
    mean_fill(data, 'T1_ACT_DAYS')
    mean_fill(data, 'R3_ACT_DAYS')
    mean_fill(data, 'T0_DS_VOL')
    mean_fill(data, 'T1_DS_VOL')
    mean_fill(data, 'R3_DS_VOL')
    return data

#location_behavior
def get_location_behavior(data):
    #读取高危地
    data_gaowei = pd.read_excel('./高危地.xlsx')
    array = data.loc[:, 'ROAM_PLACE_TOP1'].values.reshape(-1, 1)
    for i in range(len(array)):
        if array[i,0] in list(data_gaowei['city_name']):
            data.iat[i, list(data.columns).index('ROAM_PLACE_TOP1')] = 1
        else:
            data.iat[i, list(data.columns).index('ROAM_PLACE_TOP1')] = 0

    data['ROAM_PLACE_DAYS_TOP1'] = MinMaxScaler(feature_range=(0, 1)).fit_transform((data[['ROAM_PLACE_DAYS_TOP1']]))
    return data


if __name__=='__main__':
    #读取训练集
    trainset = pd.read_csv('./ANTI_FRAUD2_Z_train_head.csv',index_col='User_ID',sep=',',encoding='gbk', error_bad_lines=False) # gb18030   utf-8   unicode_escape   gbk
    #删除错行导致的中文行
    if_ch(trainset, 'R3_SMS_CNTS')
    if_ch(trainset, 'R3_SMS_DAYS')
    if_ch(trainset, 'T0_SMS_CNTS')
    if_ch(trainset, 'T0_SMS_DAYS')
    if_ch(trainset, 'slient_days')
    if_ch(trainset, 't0_ONLINE_DAY')
    if_ch(trainset, 't1_ONLINE_DAY')
    if_ch(trainset, 'R3_ONLINE_DAY')
    if_ch(trainset, 'ROAM_PLACE_CNT')
    isIntSeriously(trainset,'ROAM_PLACE_CNT')
    if_ch(trainset,'ROAM_PLACE_DAYS_TOP1')
    # 读取标签列和部分特征
    trainset\
        =trainset[['Is_cheat','Stat_Name','AGE','ID_NBR_CITY','CHANNEL_LVL1_NAME_2014','REAL_CUST_FLG','SAME_ID_ASSETS_FLG','innet_month','MKT_STAR_GRADE_NAME','ZK_FLG','QQW_FLG','TEAM_FLG','CDMA_CNT','ITV_CNT','FIX_CNT','SAME_ID_C_CNTS','WS_ID_FLG' #用户基本信息
        ,'is_agree_flg','agree_exp_flg','prom_amt','T0_FIN_AMT','T1_FIN_AMT','R3_FIN_AMT','PROM_T0_FIN_AMT','PAY_MODE_CD','INV_AMT_FLG' #合约、套餐、消费行为
        ,'MAXDAY_CALL_CNT','OPP_NBRS','OPP_NBRS_CTC_PCT','CALL_DIS','O_Call_Cnt','O_Call_Dur','T0_CALL_DAYS','T0_CALL_CNT','T0_CALL_DUR' ,'T1_CALL_DAYS','T1_CALL_CNT','T1_CALL_DUR','R3_CALL_DAYS','R3_CALL_CNT','R3_CALL_DUR','T0_O_AVG_CALL','T0_O_CALL_60','T0_O_CALL_120','T0_O_CALL_300','T0_O_CALL_300s','FIRST_CALL_PLACE' #通话行为
        ,'R3_SMS_CNTS','R3_SMS_DAYS' ,'T0_SMS_CNTS','T0_SMS_DAYS' #短信行为
        ,'slient_days','t0_ONLINE_DAY','t1_ONLINE_DAY','R3_ONLINE_DAY','T0_ACT_DAYS','T1_ACT_DAYS','R3_ACT_DAYS','T0_DS_VOL','T1_DS_VOL','R3_DS_VOL' #用户上网行为
        ,'ROAM_PLACE_CNT','ROAM_PLACE_TOP1','ROAM_PLACE_DAYS_TOP1']] #位置信息
    #处理训练集数据
    print(trainset)
    get_base_info(trainset)
    get_consume_behavior(trainset)
    get_calling_behavior(trainset)
    get_sms_behavior(trainset)
    get_internet_behavior(trainset)
    get_location_behavior(trainset)

    #利用皮尔森系数选择与标签相关性较大的特征
    col=[]
    prr=[]
    for i in range(len(list(trainset.columns))-1):
        cc_corr=trainset.iloc[:,[0,i+1]].corr(method='pearson')
        if len(cc_corr.values.reshape(-1, 1))==4:
            if abs(cc_corr.values.reshape(-1, 1)[1,0])>=0.1 and abs(cc_corr.values.reshape(-1, 1)[1,0])!='nan':
                col.append(list(trainset.iloc[:,[0,i+1]].columns)[1])
                prr.append(abs(cc_corr.values.reshape(-1, 1)[1,0]))
    #确定最终的训练集
    smo=SMOTE(random_state=42)
    X_smo ,y_smo= smo.fit_sample(trainset[col],trainset['Is_cheat'])
    dt=datetime.now()
    print(dt.strftime( '%Y-%m-%d %H:%M:%S' )+' 训练集处理完成')


    #读取测试集
    testset = pd.read_csv('./test.csv', index_col='User_ID', sep=',', encoding='utf-8')
    # 删除错行导致的中文行
    # if_ch(testset, 'R3_SMS_CNTS')
    # if_ch(testset, 'R3_SMS_DAYS')
    # if_ch(testset, 'T0_SMS_CNTS')
    # if_ch(testset, 'T0_SMS_DAYS')
    # if_ch(testset, 'slient_days')
    # if_ch(testset, 't0_ONLINE_DAY')
    # if_ch(testset, 't1_ONLINE_DAY')
    # if_ch(testset, 'R3_ONLINE_DAY')
    # if_ch(testset, 'ROAM_PLACE_CNT')
    # isIntSeriously(testset, 'ROAM_PLACE_CNT')
    # if_ch(testset, 'ROAM_PLACE_DAYS_TOP1')
    # 处理训练集数据
    get_base_info(testset)
    get_consume_behavior(testset)
    get_calling_behavior(testset)
    get_sms_behavior(testset)
    get_internet_behavior(testset)
    get_location_behavior(testset)
    testset = testset[col]
    dt = datetime.now()
    print(dt.strftime('%Y-%m-%d %H:%M:%S') + ' 测试集处理完成')

    #随机森林调参
    # scorel=[]
    # for i in range(97,104,1):
    #     rfc=RandomForestClassifier(n_estimators=i,random_state=90,n_jobs=-1)
    #     score = cross_val_score(rfc,X_smo,y_smo, cv=10,scoring='f1').mean()
    #     scorel.append(score)
    #     dt = datetime.now()
    #     print(dt.strftime( '%Y-%m-%d %H:%M:%S' ),i,score)
    # print(max(scorel),scorel.index(max(scorel))+97)


    # scorel = []
    # for i in range(0, 25, 5):
    #     rfc = RandomForestClassifier(n_estimators=100, random_state=90, n_jobs=-1, max_depth=i+1)
    #     score = cross_val_score(rfc, X_smo, y_smo, cv=10,scoring='f1').mean()
    #     scorel.append(score)
    #     dt = datetime.now()
    #     print(dt.strftime('%Y-%m-%d %H:%M:%S'), i+1 , score)
    # print(max(scorel), scorel.index(max(scorel))*5 +1)

    """
    scorel = []
    for i in range(0, 160, 20):
        rfc = RandomForestClassifier(n_estimators=101, random_state=90, n_jobs=-1, max_depth=35, min_samples_leaf=i+1 )
        score = cross_val_score(rfc, X_smo, y_smo, cv=10).mean()
        scorel.append(score)
        dt = datetime.now()
        print(dt.strftime('%Y-%m-%d %H:%M:%S'), i+1 , score)
    print(max(scorel), scorel.index(max(scorel)) * 20 + 1)
    """
    """
    scorel = []
    for i in range(50, 201, 20):
        rfc = RandomForestClassifier(n_estimators=101, random_state=90, n_jobs=-1, max_depth=35, min_samples_leaf= 1,min_samples_split=i)
        score = cross_val_score(rfc, X_smo, y_smo, cv=5).mean()
        scorel.append(score)
        dt = datetime.now()
        print(dt.strftime('%Y-%m-%d %H:%M:%S'), i , score)
    print(max(scorel), scorel.index(max(scorel))  + 50)
      """
    """
    scorel = []
    for i in range(152, 300, 10):
        rfc = RandomForestClassifier(n_estimators=101, random_state=90, n_jobs=-1, max_depth=35, min_samples_leaf= 1,min_samples_split=2,max_leaf_nodes=i)
        score = cross_val_score(rfc, X_smo, y_smo, cv=5).mean()
        scorel.append(score)
        dt = datetime.now()
        print(dt.strftime('%Y-%m-%d %H:%M:%S'), i , score)
    print(max(scorel), scorel.index(max(scorel)) *10 +152)
    """
    """
    scorel = []
    for i in range(0, 46, 1):
        rfc = RandomForestClassifier(n_estimators=101, random_state=90, n_jobs=-1, max_depth=35, min_samples_leaf=1,
                                     min_samples_split=2
                                     #, max_leaf_nodes=i
                                     ,max_features=i+1 )
        score = cross_val_score(rfc, X_smo, y_smo, cv=5).mean()
        scorel.append(score)
        dt = datetime.now()
        print(dt.strftime('%Y-%m-%d %H:%M:%S'), i+1, score)
    print(max(scorel), scorel.index(max(scorel))  + 1)
    """
    """
    scorel = []
    for i in ['gini','entropy']:
        rfc = RandomForestClassifier(n_estimators=101, random_state=90, n_jobs=-1, max_depth=35, min_samples_leaf=1,
                                     min_samples_split=2
                                     # , max_leaf_nodes=i
                                     , max_features=17
                                     ,criterion=i
                                     )
        score = cross_val_score(rfc, X_smo, y_smo, cv=5).mean()
        scorel.append(score)
        dt = datetime.now()
        print(dt.strftime('%Y-%m-%d %H:%M:%S'), i , score)
    print(max(scorel), scorel.index(max(scorel)))
    """


    #训练集和测试集
    train_data, test_data, train_target, test_target = train_test_split(X_smo, y_smo, test_size=0.3
                                                                        , random_state=1)
    rfc = RandomForestClassifier(criterion='gini'
                                 , n_estimators=100
                                 , random_state=90
                                 , n_jobs=-1
                                 , max_depth=22
                                 , min_samples_leaf=1
                                 , min_samples_split=2
                                 , max_features='sqrt'
                                 ,oob_score=True
                                 )
    rfc.fit(train_data, train_target)
    #trainscore=rfc.score(train_data, train_target)
    testscore = rfc.score(trainset[col], trainset['Is_cheat'])
    #y_train_pred=rfc.predict(train_data)
    y_pred=rfc.predict(trainset[col])
    #trainset[col],trainset['Is_cheat'])
    #auc=roc_auc_score(test_target,y_pred)
    #auc1=roc_auc_score(train_target,y_train_pred)
    f1=f1_score(trainset['Is_cheat'],y_pred,average='binary')
    #oob_score=rfc.oob_score_

    #x=pd.DataFrame({'User_ID': testset.index, 'Is_cheat': np.array([int(i) for i in y_pred])},
                 #index=testset.index)
    #y=pd.concat([x,test_target],axis=1)
    #x.to_csv('./222.csv')
    # print('模型在训练集集上拟合度{:.2f}%'.format(trainscore * 100))
    print('模型在测试集上拟合度{:.2f}%'.format(testscore * 100))
    # print('模型袋外分数{:.2f}%'.format(oob_score * 100))
    # print('auc {:.2f}%'.format(auc * 100))
    # print('auc1 {:.2f}%'.format(auc1 * 100))
    print('f1 {:.2f}%'.format(f1 * 100))
    # print('模型各参数重要程度{:}'.format(np.array([*zip(X_smo.columns[:], rfc.feature_importances_)]).reshape(-1, 2)))






b'Skipping line 10: expected 87 fields, saw 88\nSkipping line 15: expected 87 fields, saw 88\nSkipping line 24: expected 87 fields, saw 88\nSkipping line 42: expected 87 fields, saw 88\nSkipping line 49: expected 87 fields, saw 88\nSkipping line 56: expected 87 fields, saw 88\nSkipping line 67: expected 87 fields, saw 88\nSkipping line 68: expected 87 fields, saw 88\nSkipping line 75: expected 87 fields, saw 88\nSkipping line 79: expected 87 fields, saw 88\nSkipping line 84: expected 87 fields, saw 88\nSkipping line 137: expected 87 fields, saw 88\nSkipping line 154: expected 87 fields, saw 88\nSkipping line 191: expected 87 fields, saw 88\nSkipping line 193: expected 87 fields, saw 88\nSkipping line 254: expected 87 fields, saw 88\nSkipping line 259: expected 87 fields, saw 91\nSkipping line 270: expected 87 fields, saw 88\nSkipping line 308: expected 87 fields, saw 88\nSkipping line 310: expected 87 fields, saw 89\nSkipping line 325: expected 87 fields, saw 88\nSkipping line 381: exp

b'Skipping line 8585: expected 87 fields, saw 88\nSkipping line 8603: expected 87 fields, saw 88\nSkipping line 8619: expected 87 fields, saw 88\nSkipping line 8650: expected 87 fields, saw 88\nSkipping line 8655: expected 87 fields, saw 88\nSkipping line 8657: expected 87 fields, saw 88\nSkipping line 8661: expected 87 fields, saw 88\nSkipping line 8679: expected 87 fields, saw 88\nSkipping line 8777: expected 87 fields, saw 88\nSkipping line 8786: expected 87 fields, saw 88\nSkipping line 8789: expected 87 fields, saw 88\nSkipping line 8805: expected 87 fields, saw 88\nSkipping line 8819: expected 87 fields, saw 89\nSkipping line 8821: expected 87 fields, saw 89\nSkipping line 8883: expected 87 fields, saw 90\nSkipping line 8895: expected 87 fields, saw 88\nSkipping line 8913: expected 87 fields, saw 88\nSkipping line 8943: expected 87 fields, saw 89\nSkipping line 8951: expected 87 fields, saw 88\nSkipping line 8969: expected 87 fields, saw 89\nSkipping line 8995: expected 87 fields

b'Skipping line 17112: expected 88 fields, saw 89\nSkipping line 17324: expected 88 fields, saw 89\nSkipping line 17401: expected 88 fields, saw 89\nSkipping line 17551: expected 88 fields, saw 90\nSkipping line 19547: expected 88 fields, saw 89\nSkipping line 19699: expected 88 fields, saw 89\nSkipping line 20044: expected 88 fields, saw 89\nSkipping line 20574: expected 88 fields, saw 89\nSkipping line 20730: expected 88 fields, saw 89\nSkipping line 20839: expected 88 fields, saw 89\nSkipping line 20960: expected 88 fields, saw 89\nSkipping line 21066: expected 88 fields, saw 89\nSkipping line 21117: expected 88 fields, saw 89\nSkipping line 21162: expected 88 fields, saw 89\nSkipping line 21197: expected 88 fields, saw 89\nSkipping line 21373: expected 88 fields, saw 89\nSkipping line 21462: expected 88 fields, saw 90\nSkipping line 21772: expected 88 fields, saw 89\nSkipping line 21809: expected 88 fields, saw 89\nSkipping line 22539: expected 88 fields, saw 89\nSkipping line 2261

b'Skipping line 58537: expected 88 fields, saw 89\nSkipping line 58961: expected 88 fields, saw 90\nSkipping line 59384: expected 88 fields, saw 90\nSkipping line 59392: expected 88 fields, saw 89\nSkipping line 59400: expected 88 fields, saw 89\nSkipping line 59767: expected 88 fields, saw 89\nSkipping line 60009: expected 88 fields, saw 89\nSkipping line 60101: expected 88 fields, saw 90\nSkipping line 60442: expected 88 fields, saw 89\nSkipping line 60720: expected 88 fields, saw 92\nSkipping line 60971: expected 88 fields, saw 89\nSkipping line 61009: expected 88 fields, saw 89\nSkipping line 61068: expected 88 fields, saw 89\nSkipping line 61085: expected 88 fields, saw 90\nSkipping line 61169: expected 88 fields, saw 90\nSkipping line 61457: expected 88 fields, saw 90\nSkipping line 61519: expected 88 fields, saw 89\nSkipping line 61836: expected 88 fields, saw 89\nSkipping line 61917: expected 88 fields, saw 89\nSkipping line 62060: expected 88 fields, saw 89\nSkipping line 6234

b'Skipping line 99406: expected 88 fields, saw 89\nSkipping line 100052: expected 88 fields, saw 89\nSkipping line 100247: expected 88 fields, saw 89\nSkipping line 100542: expected 88 fields, saw 89\nSkipping line 100597: expected 88 fields, saw 89\nSkipping line 100816: expected 88 fields, saw 89\nSkipping line 101682: expected 88 fields, saw 89\nSkipping line 101978: expected 88 fields, saw 89\n'


TypeError: invalid type comparison